# Probit Regressions

---
## Experimental Setup

### Libraries & Settings

In [1]:
import os              # General OS commands
import numpy as np     # NumPy
import pandas as pd    # Python Data Analysis Library
import zipfile         # Compress/decompress ZIP files
import sqlite3         # SQLite3 Database Driver
import re              # Regular Expressions

In [2]:
import statsmodels.api as sm
from sklearn import metrics
import math

In [3]:
# Never truncate columns, display all the data
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', -1)

# Display floating-point numbers with 4 decimals in `pandas.DataFrame`
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt
# Display MatPlotLib stuff inline
%matplotlib inline

### Database

In [4]:
zip_filename = "../../data/ee-insee-2005_custom-sqlite.zip"
eedb = zip_filename.replace("-sqlite.zip", ".sqlite")

if not os.path.exists(eedb):
    with zipfile.ZipFile(zip_filename) as zip_file:
        zip_file.extractall("../../data/")

In [5]:
with sqlite3.connect(eedb) as con:
    query = "SELECT * FROM eec05_custom"
    eec05 = pd.read_sql_query(query, con)

In [6]:
# # Create a ("female" x "enfant") interaction variable
# bool_ = eec05[["enfants_", "female_"]].astype(bool)
# eec05["female_enfants_"] = (bool_.enfants_ & bool_.female_).astype(int)

# # Drop data we don't need
# eec05 = eec05[eec05.age60_ == 0]
# eec05 = eec05.drop("age60_", 1)

---
## Regression

In [7]:
# Build a list of parameters to include in the model, using regex
# https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
filters = {
#     "trim": "^trim$",
    "age": "^age$",
    "female": "^female_$",
    
#     "nationalite": "^nat28_[0-9]*_$",
#     "matri": "^matri_[0-9]*_$",
#     "enfants": "^nbenf18_[0-9]*_$",
    
#     "diploma": "^dip_[0-9]*_$",
#     "specialite": "^spe_[0-9]*_$",
    
#     "region": "^reg_[0-9]*_$",
    "unite_urbaine": "^tur5_[0-9]*_",
    
#     "csp_pere": "^cspp_[0-9]*_$",
#     "csp": "^cstot_[0-9]*_$"
}

params = {k: sorted([x for x in eec05.columns if re.match(r, x)]) for (k, r) in filters.items()}

# Avoid the dummy variable trap
params = {k: (v if len(v) == 1 else v[:-1]) for (k, v) in params.items()}
# params["region"] += ["region2_"]

In [8]:
trims = ["t{}".format(x) for x in sorted(eec05["trim"].unique())]
eec05_ = {t: eec05[eec05.trim == int(t[1])] for t in trims}
X = {t: eec05_[t][sum(params.values(), [])] for t in eec05_}
y = {t: eec05_[t]["actop_"] for t in eec05_}

### Train the model using `trim=1` data

In [9]:
# Fit using `statsmodels`
reg_logit_sm = sm.Probit(y["t1"], sm.add_constant(X["t1"])).fit(disp=False)

# Print the (`statsmodels`) model summary
reg_logit_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                 actop_   No. Observations:                71800
Model:                         Probit   Df Residuals:                    71793
Method:                           MLE   Df Model:                            6
Date:                Mon, 27 Feb 2017   Pseudo R-squ.:                 0.06704
Time:                        12:12:00   Log-Likelihood:                -46385.
converged:                       True   LL-Null:                       -49718.
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -1.0070      0.017    -57.583      0.000        -1.041    -0.973
age            0.0187      0.000     73.361      0.000         0.018     0.019
tur5_1_       -0.0148      0.017     -0.894      0.371        -0.047     0.018
tur5_2_        0.0869      0.018      4.950      0.000         0.053     0.121
tur5_3_        0.1490      0.016      9.224      0.000         0.117     0.181
tur5_4_        0.1513      0.016      9.349      0.000         0.120     0.183
female_        0.2441      0.010     25.315      0.000         0.225     0.263
==============================================================================
"""

In [10]:
# Create a DataFrame of coefficients
regressors = list(X["t1"].columns) + ["const"]
coeffs = pd.DataFrame(index=regressors)
coeffs["sm"] = reg_logit_sm.params

# Note: in the Logit notebook, the purpose of this DataFrame was to compare the results
# from the `sklearn` and `statsmodels` regressions – we cannot do that here since `sklearn`
# cannot perform Probit regressions – we keep the `coeffs` variable for consistency

In [11]:
# Check the accuracy of the model on the training set
predicted = np.where(reg_logit_sm.predict() < 0.5, 0, 1)
print "Accuracy: {}".format(metrics.accuracy_score(y["t1"], predicted))

Accuracy: 0.614679665738


### Apply the model to test data (`trim=2/3/4`)

In [12]:
# Check the accuracy of the model on the training & test sets

for t in trims:
#     predicted = np.where(reg_logit_sk.predict_proba(X[t]).T[1] < 0.5, 0, 1)
#     predicted = reg_logit_sk.predict(X[t])
    predicted = np.where(reg_logit_sm.predict(sm.add_constant(X[t])) < 0.5, 0, 1)
    print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y[t], predicted))

# Note: the three methods for computing `predicted` are equivalent (THRESHOLD = 0.5)

Accuracy (t1): 0.614679665738
Accuracy (t2): 0.620379648578
Accuracy (t3): 0.627715443468
Accuracy (t4): 0.625363717554


---
## Marginal Effects

In [13]:
marginal_effects = pd.DataFrame()

### Analytically (using the derivative formula)
Recall that:
$$EM(x_1) = \frac{\partial{\mathbb{E}[y|x]}}{\partial{x_1}} = \beta_1 \Phi'(\beta_0 + \beta_1 x_1 + ... ) = \beta_1 \phi(\beta_0 + \beta_1 x_1 + ... )$$
where
$$\Phi(x^T\beta) = P(y=1|x) = \mathbb{E}[y|x]$$
is the *cumulative distribution function* and
$$\Phi'(x) = \phi(x) = \frac{1}{\sqrt{2\pi}}\, e^{-\frac{x^2}{2}}$$
the *probability density function* of a **standard** normal distribution ($\mu = 0, \sigma = 1$)

**Note**: for each parameter, we calculate the *mean* marginal effect over the entire training dataset

In [14]:
def phi(x):
    return 1/math.sqrt(2*math.pi) * math.exp(-x**2/2)

In [15]:
y_fitted = X["t1"].dot(coeffs["sm"][:-1])
y_fitted += coeffs["sm"]["const"]
dlambda_y = y_fitted.map(phi)

In [16]:
marginal_effects["analytical"] = coeffs["sm"]*dlambda_y.mean()

### By "rule-of-thumb"
Note (from above) that:
$$EM(x_1) = \beta_1 \phi(\beta_0 + \beta_1 x_1 + ... )$$
and 
$$\max \phi(x) = \phi(0) = \frac{1}{\sqrt{2\pi}} \approx \frac{1}{2.5}$$
Therefore, we have that:
$$EM(x_1) \approx \frac{\beta_1}{2.5}$$

The "rule-of-thumb" thus divides all regression coefficients by 2.5 to approximate the marginal effect.

In [17]:
marginal_effects["rule_of_thumb"] = coeffs["sm"]/2.5

### By modifying the dataset ("passage de tout le monde en licence")

In [18]:
brute_force = pd.Series()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1

#         proba_one = reg_logit_sk.predict_proba(X_one).T[1].mean()
#         proba_t1 = reg_logit_sk.predict_proba(X["t1"]).T[1].mean()
        proba_one = reg_logit_sm.predict(sm.add_constant(X_one, has_constant="add")).mean()
        proba_t1 = reg_logit_sm.predict(sm.add_constant(X["t1"])).mean()
        brute_force[pivot] = proba_one - proba_t1

In [19]:
marginal_effects["brute_force"] = brute_force
marginal_effects.transpose()

,age,tur5_1_,tur5_2_,tur5_3_,tur5_4_,female_,const
analytical,0.0069,-0.0054,0.0321,0.0550,0.0559,0.0901,-0.3719
rule_of_thumb,0.0075,-0.0059,0.0348,0.0596,0.0605,0.0976,-0.4028
brute_force,-0.3023,-0.0361,0.0015,0.0244,0.0253,0.0432,nan


In [20]:
ratios = marginal_effects.copy()
ratios = ratios.div(ratios["analytical"], axis=0)
ratios.transpose()

,age,tur5_1_,tur5_2_,tur5_3_,tur5_4_,female_,const
analytical,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
rule_of_thumb,1.0831,1.0831,1.0831,1.0831,1.0831,1.0831,1.0831
brute_force,-43.8046,6.6303,0.0479,0.4441,0.4524,0.4789,nan
